# Data Importing & Cleaning

In [ ]:
# Data Importing
import pandas as pd

data = pd.read_csv('../input/arabic-idioms-dataset/Dataset.csv', sep=',')
data

In [ ]:
data['class'].value_counts()

In [ ]:
data['target'].value_counts()

# Data Modeling

In [ ]:
X = data[['class']] 
y = data['target']
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

print('Training Data Shape:', X_train.shape)
print('Testing Data Shape: ', X_test.shape)

In [ ]:
from sklearn.linear_model import LogisticRegression

lr_model = LogisticRegression(solver='lbfgs')

lr_model.fit(X_train, y_train)

In [ ]:
from sklearn import metrics

predictions = lr_model.predict(X_test)

print(metrics.confusion_matrix(y_test,predictions))

print(metrics.classification_report(y_test,predictions))

print(metrics.accuracy_score(y_test,predictions))

In [ ]:
from sklearn.naive_bayes import MultinomialNB
nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)
predictions = nb_model.predict(X_test)
print(metrics.confusion_matrix(y_test,predictions))
print(metrics.classification_report(y_test,predictions))
print(metrics.accuracy_score(y_test,predictions))

In [ ]:
from sklearn.svm import SVC
svc_model = SVC(gamma='auto')
svc_model.fit(X_train,y_train)

predictions = svc_model.predict(X_test)
print(metrics.confusion_matrix(y_test,predictions))
print(metrics.classification_report(y_test,predictions))
print(metrics.accuracy_score(y_test,predictions))

In [ ]:
from sklearn.model_selection import train_test_split

X = data['idiom']  
y = data['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
y_train.value_counts(), y_test.value_counts()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()

X_train_counts = count_vect.fit_transform(X_train)
X_train_counts.shape

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()

X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

In [ ]:
pd.DataFrame(X_train_tfidf)[0]

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

X_train_tfidf = vectorizer.fit_transform(X_train) # remember to use the original X_train set
X_train_tfidf.shape

In [ ]:
from sklearn.svm import LinearSVC
clf = LinearSVC()
clf.fit(X_train_tfidf,y_train)

In [ ]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', clf),])


text_clf.fit(X_train, y_train)  


predictions = text_clf.predict(X_test)
from sklearn import metrics
print(metrics.confusion_matrix(y_test,predictions))

In [ ]:
print(metrics.classification_report(y_test,predictions)) 

In [ ]:
print(metrics.accuracy_score(y_test,predictions))

In [ ]:
text_clf.predict(['السوق السوداء'])[0]

# Feature Engineering

In [ ]:
data

In [ ]:
from nltk.tokenize import word_tokenize

# Adding number of words as one of our features
data["N"] = [len(word_tokenize(i)) for i in data["idiom"]]
data

In [ ]:
X = data['idiom']  
y = data['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=70)

count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)

tfidf_transformer = TfidfTransformer()

X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape


vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train) # remember to use the original X_train set
X_train_tfidf.shape

clf = LinearSVC()
clf.fit(X_train_tfidf,y_train)

In [ ]:
pipeline = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', LinearSVC()),])


pipeline.fit(X_train, y_train)  


predictions = pipeline.predict(X_test)
from sklearn import metrics
print(metrics.confusion_matrix(y_test,predictions))

In [ ]:
import csv
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, LSTM, Dropout, Activation, Embedding, Bidirectional,SimpleRNN
import nltk

In [ ]:
vocab_size = 5000
embedding_dim = 64
max_length = 200
oov_tok = '<OOV>' #  Out of Vocabulary
training_portion = 0.8
articles = data['idiom']
labels = data['target']

len(labels),len(articles)

In [ ]:
labels[0],articles[0]

In [ ]:
train_size = int(len(articles) * training_portion)

train_articles = articles[0: train_size]
train_labels = labels[0: train_size]

validation_articles = articles[train_size:]
validation_labels = labels[train_size:]

In [ ]:
print('train_articles' ,len(train_articles))
print('train_labels', len(train_labels))
print('validation_articles', len(validation_articles))
print('validation_labels', len(validation_labels))

In [ ]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_articles)
word_index = tokenizer.word_index

In [ ]:
word_index

In [ ]:
tokenizer.texts_to_sequences(['كتب الولد الواجب.'])


In [ ]:
train_sequences = tokenizer.texts_to_sequences(train_articles)
len(train_sequences)

In [ ]:
train_sequences[0]


In [ ]:
train_padded = pad_sequences(train_sequences, maxlen=max_length,)
len(train_padded),len(train_padded[0])

In [ ]:
validation_sequences = tokenizer.texts_to_sequences(validation_articles)
validation_padded = pad_sequences(validation_sequences, maxlen=max_length)
set(labels)